In [ ]:
# Contribution analysis for all activities in dfresults
# This will analyze the contributions of upstream processes to each activity's impact

contribution_results = {}

# Get all activity codes from all_acts_fu (which matches dfresults)
activity_codes = list(all_acts_fu.keys())

print(f"Performing contribution analysis for {len(activity_codes)} activities...")

# Perform contribution analysis for each activity and each method
for act_code in activity_codes:
    contribution_results[act_code] = {}
    
    # Get the activity object from the database
    act = None
    for activity in paw_db:
        if activity["code"] == act_code:
            act = activity
            break
    
    if act is None:
        print(f"Warning: Activity {act_code} not found in database")
        continue
    
    # Create functional unit for this activity (use the same FU structure as in all_acts_fu)
    fu = all_acts_fu[act_code]
    
    # Perform contribution analysis for each method
    for method in ef_methods:
        # Create LCA calculation for this activity and method
        lca = bc.LCA(fu, method)
        lca.lci()
        lca.lcia()
        
        # Perform contribution analysis
        ca = bwa.contribution.ContributionAnalysis(lca)
        
        # Get sorted contributions
        sorted_contributions = ca.sort_array(limit_type='percent', limit=0.01)  # Show contributions >= 1%
        
        contribution_results[act_code][method] = sorted_contributions

# Display contribution analysis results
print("=" * 80)
print("CONTRIBUTION ANALYSIS FOR ALL ACTIVITIES")
print("=" * 80)

for act_code, methods_dict in contribution_results.items():
    print(f"\n{'='*80}")
    print(f"Activity: {act_code}")
    print(f"{'='*80}")
    
    for method, contributions in methods_dict.items():
        method_name = f"{method[1]} - {method[2]}" if len(method) > 2 else str(method)
        print(f"\nMethod: {method_name}")
        print("-" * 80)
        
        if contributions is not None and len(contributions) > 0:
            # Convert to DataFrame for better display
            contrib_df = pd.DataFrame(contributions)
            print(contrib_df.head(20))  # Show top 20 contributors
        else:
            print("No significant contributions found or analysis failed")

# Store contribution results in a more accessible format
contribution_data = {}

for act_code, methods_dict in contribution_results.items():
    contribution_data[act_code] = {}
    for method, contributions in methods_dict.items():
        if contributions is not None and len(contributions) > 0:
            contribution_data[act_code][method] = pd.DataFrame(contributions)
        else:
            contribution_data[act_code][method] = pd.DataFrame()

print("\n" + "=" * 80)
print("Contribution analysis complete!")
print("=" * 80)
print(f"\nAccess contribution data via: contribution_data['activity_code']['method']")
print(f"Example: contribution_data['{list(contribution_data.keys())[0] if contribution_data else 'act_code'}'][ef_methods[0]]")
